In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import nltk

In [4]:
from gensim.models import Word2Vec

In [5]:
data = pd.read_csv("/content/tweets.csv")

In [6]:
data.info

<bound method DataFrame.info of         id  label                                              tweet
0        1      0  #fingerprint #Pregnancy Test https://goo.gl/h1...
1        2      0  Finally a transparant silicon case ^^ Thanks t...
2        3      0  We love this! Would you go? #talk #makememorie...
3        4      0  I'm wired I know I'm George I was made that wa...
4        5      1  What amazing service! Apple won't even talk to...
...    ...    ...                                                ...
7915  7916      0  Live out loud #lol #liveoutloud #selfie #smile...
7916  7917      0  We would like to wish you an amazing day! Make...
7917  7918      0  Helping my lovely 90 year old neighbor with he...
7918  7919      0  Finally got my #smart #pocket #wifi stay conne...
7919  7920      0  Apple Barcelona!!! #Apple #Store #BCN #Barcelo...

[7920 rows x 3 columns]>

In [9]:
data.head(3)

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...


In [8]:
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [10]:
import string

In [13]:
import string
import re
def remove_punctuation_of_text(text):
  punctuation_free = ''.join([i for i in text if i not in string.punctuation])
  #replacing links with empty string using regex
  no_links = re.sub(r'http\S+','',punctuation_free)
  return no_links

In [14]:
nltk.download('punkt')
def tokenization(text):
  tokens = nltk.word_tokenize(text)
  return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
nltk.download('stopwords')
stop_words= nltk.corpus.stopwords.words('english')
def remove_stop_words(text):
  no_stop_words = [i for i in text if i not in stop_words]
  return no_stop_words

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lem = WordNetLemmatizer()
def lemmatized_text(text):
  lemma = [wordnet_lem.lemmatize(word) for word in text]
  return lemma

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [16]:
def preprocess(column):
  corpus = []
  for item in column:
    new_item = remove_punctuation_of_text(item)
    new_item = new_item.lower()
    new_item = tokenization(new_item)
    new_item = remove_stop_words(new_item)
    new_item = lemmatized_text(new_item)
    corpus.append(new_item)
  return corpus

In [21]:
data['processed'] = preprocess(data['tweet'])

In [17]:
pd.set_option('display.max_colwidth',None)
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [ ]:
#Bagofwords

In [20]:
from sklearn.model_selection import train_test_split
import gensim


In [22]:
x = data['processed']
y = data['label']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [23]:
word2vec_model = gensim.models.Word2Vec(x_train, min_count=1)

In [24]:
words = word2vec_model.wv.index_to_key

In [25]:
train_vec = np.array([np.array(word2vec_model.wv[i] for i in item if i in words) for item in x_train], dtype=object)
test_vec = np.array([np.array(word2vec_model.wv[i] for i in item if i in words) for item in x_test], dtype=object)


In [26]:
train_vec.ndim

1

In [27]:
x_train_vec_avg = [np.mean(v, axis=0) if v.ndim > 0 and v.size > 0 else np.zeros(100, dtype=float) for v in train_vec]
x_test_vec_avg = [np.mean(v, axis=0) if v.ndim > 0 and v.size > 0 else np.zeros(100, dtype=float) for v in test_vec]

In [28]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(x_train_vec_avg, y_train)

RandomForestClassifier()

In [29]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(x_train_vec_avg)
accuracy_score(y_pred,y_train)

0.7484217171717171

In [30]:
y_pred = clf.predict(x_test_vec_avg)
accuracy_score(y_pred,y_test)

0.7272727272727273

In [32]:
sms = "What a surprise, you won one million dollars!"
sms = preprocess(sms)


valid_words = [word2vec_model.wv[i] for i in sms if i in words]
x_test1 = np.array(valid_words).mean(axis=0) if valid_words else np.zeros(100, dtype=float)
x_test1 = x_test1.reshape(1, -1) if not np.isnan(x_test1).any() else np.zeros((1, 100), dtype=float)

y_pred = clf.predict(x_test1)
print(y_pred)

[0]


In [33]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,2))
corpus = [' '.join(doc) for doc in preprocess(data['tweet'])]
train_data = cv.fit_transform(corpus)
x= train_data
y = data['label']

In [34]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.2)
clf.fit(x_train,y_train)

RandomForestClassifier()

In [35]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(x_test)
accuracy_score(y_test,y_pred)

0.8465909090909091

In [ ]:
#tokenization

In [36]:
from keras.preprocessing import text
tokenizer = text.Tokenizer()

tokenizer.fit_on_texts(list(data['tweet']))

tokenized_text = tokenizer.texts_to_sequences(data['tweet'])

In [37]:
len(data['tweet'][0])

128

In [38]:
len(tokenized_text[0])

16

In [39]:
from keras.utils import pad_sequences
x = pad_sequences(tokenized_text, maxlen=100)

In [40]:
tokenizer.word_index
len(tokenizer.word_index)

24171

In [41]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, data['label'].values, test_size=0.2)

In [42]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, SimpleRNN, Dropout
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim = 128, input_length=100))
model.add(LSTM(10))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [43]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          3094016   
                                                                 
 lstm (LSTM)                 (None, 10)                5560      
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense (Dense)               (None, 50)                550       
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 3100177 (11.83 MB)
Trainable params: 31001

In [44]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
179/179 [==============================] - 24s 108ms/step - loss: 0.4381 - accuracy: 0.8024 - val_loss: 0.2655 - val_accuracy: 0.8912
Epoch 2/10
179/179 [==============================] - 19s 107ms/step - loss: 0.2147 - accuracy: 0.9206 - val_loss: 0.2447 - val_accuracy: 0.8975
Epoch 3/10
179/179 [==============================] - 20s 113ms/step - loss: 0.1123 - accuracy: 0.9669 - val_loss: 0.3130 - val_accuracy: 0.9069
Epoch 4/10
179/179 [==============================] - 24s 133ms/step - loss: 0.0581 - accuracy: 0.9828 - val_loss: 0.3762 - val_accuracy: 0.8880
Epoch 5/10
179/179 [==============================] - 16s 87ms/step - loss: 0.0360 - accuracy: 0.9904 - val_loss: 0.4455 - val_accuracy: 0.8896
Epoch 6/10
179/179 [==============================] - 21s 116ms/step - loss: 0.0226 - accuracy: 0.9937 - val_loss: 0.6216 - val_accuracy: 0.8644
Epoch 7/10
179/179 [==============================] - 19s 109ms/step - loss: 0.0130 - accuracy: 0.9972 - val_loss: 0.6438 - val_acc

In [45]:
y_preds = model.predict(x_test)
y_preds[y_preds > 0.5] = 1
y_preds[y_preds<0.5] = 0

50/50 [==============================] - 1s 9ms/step


In [46]:
accuracy_score( y_preds,y_test)

0.8484848484848485

In [47]:
test1 = "What a surprise, you won one million dollars!"
test1 = tokenizer.texts_to_sequences([test1])
test2 = pad_sequences(test1, maxlen=100)
output = model.predict(test2)
output[output>0.5] = 1
output[output < 0.5] = 0
output

1/1 [==============================] - 0s 30ms/step


array([[0.]], dtype=float32)